In [4]:
#!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 297.3 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [1]:
import shutil
from fastapi import APIRouter, File, UploadFile
from PIL import Image
from fastapi.responses import FileResponse
from tensorflow.keras.models import Model, load_model
import matplotlib.pyplot as plt

import cv2
import keras
import io
import mlflow
import numpy as np
import os
import tensorflow as tf

2025-01-31 09:19:00.022996: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-31 09:19:00.054230: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738315140.081710   23121 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738315140.090065   23121 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-31 09:19:00.116245: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [8]:
mlops_server_uri = 'https://david-rem-jedha-final-project-mlops.hf.space'
#model_path = 'models:/lung_7_classes/2'
model_path = "runs:/73f7af9ab4be4dd1be44b4653bb2bf06/model"

mlflow.set_tracking_uri(mlops_server_uri)
keras_model = mlflow.keras.load_model(model_path)

# Vérifier si le modèle est bien chargé
print(type(keras_model))  # Doit être <class 'keras.engine.functional.Functional'> ou <class 'tensorflow.keras.Model'>

# Tester une entrée factice
#dummy_input = np.zeros((1, 256, 256, 3))  # Ajustez la taille si nécessaire
#keras_model(dummy_input)
#output = keras_model.predict(dummy_input)




#pyfunc_model = mlflow.pyfunc.load_model(model_path)

<class 'keras.src.models.sequential.Sequential'>


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 18 variables whereas the saved optimizer has 33 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [32]:
for layer in keras_model.layers:
    print(layer.name)
    
print('\n')

for layer in keras_model.layers[1].layers:
    print(layer.name)

conv2d_190
InceptionV3
conv2d_191
flatten_1
dense_1


input_layer_2
conv2d_96
batch_normalization_94
activation_94
conv2d_97
batch_normalization_95
activation_95
conv2d_98
batch_normalization_96
activation_96
max_pooling2d_4
conv2d_99
batch_normalization_97
activation_97
conv2d_100
batch_normalization_98
activation_98
max_pooling2d_5
conv2d_104
batch_normalization_102
activation_102
conv2d_102
conv2d_105
batch_normalization_100
batch_normalization_103
activation_100
activation_103
average_pooling2d_9
conv2d_101
conv2d_103
conv2d_106
conv2d_107
batch_normalization_99
batch_normalization_101
batch_normalization_104
batch_normalization_105
activation_99
activation_101
activation_104
activation_105
mixed0
conv2d_111
batch_normalization_109
activation_109
conv2d_109
conv2d_112
batch_normalization_107
batch_normalization_110
activation_107
activation_110
average_pooling2d_10
conv2d_108
conv2d_110
conv2d_113
conv2d_114
batch_normalization_106
batch_normalization_108
batch_normalization_111
ba

In [31]:
keras_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_190 (Conv2D)             │ (None, 128, 128, 3)    │             6 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ InceptionV3 (Functional)        │ (None, 2, 2, 2048)     │    21,802,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_191 (Conv2D)             │ (None, 2, 2, 128)      │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         3,591 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,269,884 (92.58 MB)

 Trainable params: 2,201,229 (8.40 MB)

 Non-trainable params: 19,867,424 (75.79 MB)

 Optimizer params: 2,201,231 (8.40 MB)

In [3]:
def get_gradcam_heatmap(keras_model, img_array, layer_name):
    #input_shape = (256, 256, 3)  # Ajustez selon votre modèle
    input_shape = img_array.shape
    inputs = tf.keras.layers.Input(shape=input_shape)
    outputs = keras_model(inputs)
    keras_model2 = Model(inputs=inputs, outputs=outputs)
    for layer in keras_model.layers:
        print(layer.name)
    grad_model = Model([keras_model2.input], [keras_model.get_layer(layer_name).output, keras_model2.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, tf.argmax(predictions[0])]
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = tf.maximum(heatmap, 0) / tf.reduce_max(heatmap)
    return heatmap

def apply_gradcam(image_path, keras_model, layer_name):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (128, 128)) # Ajustez selon votre modèle
    print(img.shape)
    img_array = np.expand_dims(img, axis=-1) # Ajustez selon votre modèle, si besoin de (x,x,3) : np.expand_dims(np.expand_dims(img, axis=-1), axis=0)
    print(img_array.shape)

    heatmap = get_gradcam_heatmap(keras_model, img_array, layer_name)

    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    superimposed_img = heatmap * 0.4 + cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

    cv2.imwrite('gradcam_image.jpg', superimposed_img)
    plt.imshow(superimposed_img)
    plt.show()


In [9]:
#from PIL import Image
#img = Image.open("./image_0007.jpg")
#img_resized = img.resize((128, 128))


apply_gradcam("./image_0007.jpg", keras_model, "conv2d_191")
#apply_gradcam("./image_0007.jpg", keras_model, "mixed10")

(128, 128)
(128, 128, 1)
conv2d_190
InceptionV3
conv2d_191
flatten_1
dense_1


ValueError: The name "input_layer_3" is used 2 times in the model. All operation names should be unique.